In [2]:
test_urls = ["http://203.64.136.248/webpacIndex.jsp",
             "https://libelis.tmu.edu.tw/webpacIndex.jsp",
             # "https://libholding.ntut.edu.tw/webpacIndex.jsp",  # 北科大：欄位名稱不是叫 ISBN
             "http://webpac.ntua.edu.tw/webpacIndex.jsp",
             "https://webpac.cute.edu.tw/webpacIndex.jsp",
             "http://lib.utaipei.edu.tw/webpac/webpacIndex.jsp",
             "http://webpac.ntub.edu.tw/webpacIndex.jsp",
             "http://libsearch.tcpa.edu.tw/webpacIndex.jsp",
             "http://192.192.231.232/webpacIndex.jsp",  # 中華科大：有些詳細書目裡面沒有表格
             # "http://webpac.knjc.edu.tw/webpacIndex.jsp",  # 康寧：不能以 ISBN 搜尋
             "http://webpac.cct.edu.tw/webpacIndex.jsp",
             "http://210.60.142.23/webpacIndex.jsp",
             "https://jinwenlib.just.edu.tw/webpacIndex.jsp",
             "http://hylib.chihlee.edu.tw/webpacIndex.jsp",
             "http://webpac.lib.vnu.edu.tw/webpacIndex.jsp",
             "https://library.uch.edu.tw/webpacIndex.jsp",
             "https://hylib.lib.must.edu.tw/webpacIndex.jsp",
             "https://hyweblib.nou.edu.tw/webpac/webpacIndex.jsp",
             "https://webpac.miaoli.gov.tw/webpacIndex.jsp",
             "http://120.106.11.155/webpacIndex.jsp",
             "http://libopac.jente.edu.tw/webpacIndex.jsp",
             "http://webpac.lib.ntcu.edu.tw/webpacIndex.jsp",
             "https://hylib.ntus.edu.tw/webpacIndex.jsp",
             "https://webpac.lib.thu.edu.tw/webpacIndex.jsp",
             "http://webpac.lib.pu.edu.tw/webpac/webpacIndex.jsp",
             "http://lib.webpac.ocu.edu.tw/webpacIndex.jsp",
             "https://book.ncue.edu.tw/webpacIndex.jsp",
             "http://library.ylccb.gov.tw/webpacIndex.jsp",
             "https://www.cycab.gov.tw/webpacIndex.jsp",
             "http://library.cabcy.gov.tw/webpacIndex.jsp",
             "http://hylib.nhu.edu.tw//webpacIndex.jsp",
             "http://webpac.cnu.edu.tw/webpacIndex.jsp",
             "http://hy.lib.feu.edu.tw/webpacIndex.jsp",
             "https://webpac2.csu.edu.tw/webpacIndex.jsp",
             "http://webpac.meiho.edu.tw/webpacIndex.jsp",
             "https://lib.niu.edu.tw/webpacIndex.jsp",
             "http://libils.fgu.edu.tw/webpacIndex.jsp",
             "http://hylib.lib.nttu.edu.tw/webpac/webpacIndex.jsp",
             "https://lib.nqu.edu.tw/webpacIndex.jsp"]

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException, TimeoutException

import pandas as pd
from bs4 import BeautifulSoup
import time

my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--start-maximized")  # 視窗最大化
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"

def webpac_jsp_crawler(url, ISBN):
    test_url = url[:]  # 測試用，輸出「現在測試的網頁」
    test_start = time.time()  # 測試用，計算程式進行時間
    
    # 初始化 driver；設定 explicit waits
    driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
    wait = WebDriverWait(driver, 30)  # 最長等待時間：30 秒；查找元素的間隔時間：0.5秒（default）

    # 進入＂搜尋主頁＂
    driver.get(url)

    try:
        # 定位＂下拉式選單＂，選擇 ISBN
        search_field = wait.until(
            EC.presence_of_element_located((By.NAME, "search_field"))
        )
        select = Select(search_field)
        select.select_by_value("ISBN")
        
        # 抓取在哪間＂圖書館＂
        org = driver.title

        # 定位＂搜尋欄＂，輸入 ISBN
        search_input = wait.until(
            EC.presence_of_element_located((By.NAME, "search_input"))
        )
        search_input.send_keys(ISBN)
        search_input.submit()

        # 依據＂div.mainCon＂，來判斷搜尋結果是「＂一筆＂」還是「＂零筆＂或＂多筆＂」
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.mainCon"))
            )
            
            # 定位＂表格＂，爬取內容
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table.order"))
            )
            tgt = pd.read_html(driver.page_source, encoding="utf-8")[-2]
            tgt["圖書館"] = org
            tgt["連結"] = driver.current_url
        except (NoSuchElementException, TimeoutException):
            if ".jsp" in url:  # 用來組合不同＂詳細書目＂的網址
                url = url.replace(url.split("/")[-1], "")

            tgt = []
            # 定位＂內嵌框架＂，切換至此內部，找尋多個＂詳細書目的網址＂
            iframe = wait.until(
                EC.presence_of_element_located((By.ID, "leftFrame"))
            )
            driver.switch_to.frame(iframe)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            tgt_urls = []
            for anchor in soup.find_all("a", "bookname"):
                tgt_urls.append(url + anchor["href"])
            
            # 進入不同的＂詳細書目＂，再定位＂表格＂，爬取內容
            for urlx in tgt_urls:
                driver.get(urlx)
                
                # 定位＂表格＂，爬取內容
                wait.until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "table.order"))
                )
                sec_tgt = pd.read_html(driver.page_source, encoding="utf-8")[-2]
                sec_tgt["圖書館"] = org
                sec_tgt["連結"] = driver.current_url
                tgt.append(sec_tgt)
            tgt = pd.concat(tgt, axis=0, ignore_index=True)
        finally:
            driver.close()
            tgt.to_csv(f"{org}.csv", encoding="big5", errors="ignore")
            test_end = time.time()  # 測試用，計算程式進行時間
            print(f"程式執行成功，執行時間：{test_end - test_start:7.4f}秒，在「{org}」")  # 測試用，計算程式進行時間
            return tgt
    except Exception as e:
        test_end = time.time()  # 測試用，計算程式進行時間
        print("="*100)
        print(f"程式執行失敗，執行時間：{test_end - test_start:7.4f}秒，在「{org}」")
        print(f"網址為：「{test_url}」")
        print(f"錯誤訊息為：「{e}」")
        print("="*100)

In [4]:
df_lst = []
for test_url in test_urls:
    xxx = 
    df_lst.append(
        webpac_jsp_crawler(
            url=test_url,
            ISBN="9789573317241")
    )


程式執行成功，執行時間： 4.5893秒，在「經國管理暨健康學院 圖書館」
   #    條碼號  館藏位置                 索書號     資料類型 目前狀態 / 到期日  架號  附件    預約  \
0  1  99502  中文書櫃      873.57 857 v.1  一般圖書/一般        書在館 NaN NaN  不可預約   
1  2  99505  中文書櫃  873.57 857 v.1 c.4  一般圖書/一般       架上遺失 NaN NaN  不可預約   
2  3  99503  中文書櫃  873.57 857 v.1 c.2  一般圖書/一般        書在館 NaN NaN  不可預約   
3  4  99504  第三書庫  873.57 857 v.1 c.3  一般圖書/一般       不可外借 NaN NaN  不可預約   

             圖書館                                            連結  
0  經國管理暨健康學院 圖書館  http://203.64.136.248/bookDetail.do?id=57928  
1  經國管理暨健康學院 圖書館  http://203.64.136.248/bookDetail.do?id=57928  
2  經國管理暨健康學院 圖書館  http://203.64.136.248/bookDetail.do?id=57928  
3  經國管理暨健康學院 圖書館  http://203.64.136.248/bookDetail.do?id=57928  
程式執行成功，執行時間： 5.6620秒，在「臺北醫學大學圖書館」
   #       條碼號         館藏地/室                 索書號      資料類型 目前狀態 / 到期日  特色館藏  \
0  1  C0090480  北醫總館/2F中文圖書區    873.57 6014-1 89  中日韓圖書/一般        書在館   NaN   
1  2  C0110970  北醫總館/2F中文圖書區    873.57 6014-1 89  中日韓圖書/一般        找尋中

程式執行成功，執行時間： 9.3690秒，在「致理科技大學圖書館」
   #     條碼號     館藏地/室              索書號     資料類型 目前狀態 / 到期日  附件    預約  \
0  1  101423  圖書總館/圖書室  873.57 6014 c.1  一般圖書/一般        書在館 NaN  不可預約   
1  2  104732  圖書總館/圖書室      873.57 6014  一般圖書/一般        書在館 NaN  不可預約   

         圖書館                                                 連結  
0  致理科技大學圖書館  http://hylib.chihlee.edu.tw/bookDetail.do?id=7...  
1  致理科技大學圖書館  http://hylib.chihlee.edu.tw/bookDetail.do?id=7...  
程式執行成功，執行時間： 9.3385秒，在「萬能科大圖書館」
   #       條碼號                館藏地/室                  索書號    特藏/用途 目前狀態 / 到期日  \
0  1  C0971865          圖書總館/2樓中文書庫  873.57 4220 v.1 c.1  一般圖書/一般        書在館   
1  2  C0821658  圖書總館/4樓密集書庫(中文7-9類)      873.57 4220 c.1  一般圖書/一般        書在館   
2  3  C0746094          圖書總館/2樓中文書庫          873.57 4220  一般圖書/一般        書在館   
3  1  C0709525          圖書總館/2樓中文書庫          873.57 4220  一般圖書/一般        書在館   

   附件  特色館藏    預約      圖書館                                                 連結  
0 NaN   NaN  不可預約  萬能科大圖書館  http://

程式執行失敗，執行時間：12.0245秒，在「仁德醫護管理專科學校」
網址為：「http://libopac.jente.edu.tw/webpacIndex.jsp」
錯誤訊息為：「'list' object has no attribute 'to_csv'」
None
程式執行成功，執行時間：10.1532秒，在「國立臺中教育大學圖書館-館藏查詢」
   #     條碼號   館藏室              索書號   資料類型 目前狀態 / 到期日    預約               圖書館  \
0  1  135025  4樓書庫  873.57 6014 v.1  圖書/一般        可外借  不可預約  國立臺中教育大學圖書館-館藏查詢   

                                                  連結  
0  http://webpac.lib.ntcu.edu.tw/bookDetail.do?id...  
程式執行成功，執行時間： 4.7066秒，在「國立臺灣體育運動大學圖書館」
   #     條碼號          館藏地/室          索書號     資料類型 目前狀態 / 到期日  附件    預約  \
0  1   88309  B209書庫/B209書庫  873.57 6014  一般圖書/一般        在館內 NaN  不可預約   
1  2   88310       圖書館/一樓書庫  873.57 6014  一般圖書/一般        在館內 NaN  不可預約   
2  3  136289       圖書館/一樓書庫  873.57 6014  一般圖書/一般        在館內 NaN  不可預約   
3  4  136290       圖書館/一樓書庫  873.57 6014  一般圖書/一般        在館內 NaN  不可預約   

             圖書館                                                 連結  
0  國立臺灣體育運動大學圖書館  https://hylib.ntus.edu.tw/bookDetail.do?id=137024  

程式執行成功，執行時間： 4.9830秒，在「嘉義縣圖書館」
    #             登錄號        館藏地/區域                        索書號    特藏類型 特定用途  \
0   1  20001001713829  嘉義縣圖書館/教育處書庫                873.57 2637     一般書   一般   
1   2  20001001751241  嘉義縣圖書館/教育處書庫              C 873.57 2637  班級借閱專書   一般   
2   3  20001001852452  嘉義縣圖書館/教育處書庫                873.57 2637     一般書   一般   
3   4  20001003390584  嘉義縣圖書館/教育處書庫                873.57 8367     一般書   一般   
4   5  20002000411985  大林鎮立圖書館/大林書庫             873.57 857 c.1     一般書   一般   
5   6  20002000429334  大林鎮立圖書館/大林書庫             873.57 857 c.2     一般書   一般   
6   7  20002000430464  大林鎮立圖書館/大林書庫                 873.57 857     一般書   一般   
7   8  20003000608000  中埔鄉立圖書館/中埔書庫           873.57 8367 v.30     一般書   一般   
8   9  20004000073864  太保市立圖書館/太保書庫  873.57 8666 v.30 c.2 2000     一般書   一般   
9  10  20004000196426  太保市立圖書館/太保書庫  873.57 8666 v.30 c.3 2000     一般書   一般   

          圖書狀況 / 到期日    註記  附件      預約     圖書館  \
0               仍在館內   NaN NaN    不可預約  嘉義縣圖書館   

程式執行成功，執行時間： 6.0222秒，在「佛光大學圖書館」
   #       條碼號                  館藏地/館藏室              索書號   資料類型 目前狀態 / 到期日  \
0  1  L0004187  佛光大學圖書館/東方語文圖書區-8大類(4F)  873.57 6014 c.2  圖書/一般        書在館   
1  2  00076362              佛光大學圖書館/心輔室  873.57 6014 c.3  圖書/一般       單位藏書   
2  3  00035839  佛光大學圖書館/東方語文圖書區-8大類(4F)      873.57 6014  圖書/一般        書在館   

   附件    預約      圖書館                                               連結  
0 NaN  不可預約  佛光大學圖書館  http://libils.fgu.edu.tw/bookDetail.do?id=23383  
1 NaN  不可預約  佛光大學圖書館  http://libils.fgu.edu.tw/bookDetail.do?id=23383  
2 NaN  不可預約  佛光大學圖書館  http://libils.fgu.edu.tw/bookDetail.do?id=23383  
程式執行成功，執行時間：19.1099秒，在「國立臺東大學 館藏查詢系統」
   #      登錄號          館藏地/室             索書號         資料類型 目前狀態 / 到期日  附件  \
0  1  C058186  知本總館/3F兒童文學書庫  ＋ 846 6014 c.6  一般圖書(兒童)/一般      可預約調借 NaN   
1  2  C058185  知本總館/3F兒童文學書庫  ＋ 846 6014 c.5  一般圖書(兒童)/一般      可預約調借 NaN   
2  3  C058184  知本總館/3F兒童文學書庫  ＋ 846 6014 c.4  一般圖書(兒童)/一般      可預約調借 NaN   
3  4  C049281  知本總館/3F兒童文

In [5]:
pd.concat(df_lst, axis=0, ignore_index=True)

,#,條碼號,館藏位置,索書號,資料類型,目前狀態 / 到期日,架號,附件,預約,圖書館,...,登錄號,館藏地,特定用途,館藏地/區域,特藏類型,圖書狀況 / 到期日,註記,編號,附件/備註,館藏地/館藏室
0,1.0,99502,中文書櫃,873.57 857 v.1,一般圖書/一般,書在館,NaN,NaN,不可預約,經國管理暨健康學院 圖書館,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,99505,中文書櫃,873.57 857 v.1 c.4,一般圖書/一般,架上遺失,NaN,NaN,不可預約,經國管理暨健康學院 圖書館,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,99503,中文書櫃,873.57 857 v.1 c.2,一般圖書/一般,書在館,NaN,NaN,不可預約,經國管理暨健康學院 圖書館,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,99504,第三書庫,873.57 857 v.1 c.3,一般圖書/一般,不可外借,NaN,NaN,不可預約,經國管理暨健康學院 圖書館,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,C0090480,NaN,873.57 6014-1 89,中日韓圖書/一般,書在館,NaN,NaN,不可預約,臺北醫學大學圖書館,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,6.0,NaN,NaN,＋ 846 6014,一般圖書(兒童)/一般,可預約調借,NaN,NaN,/ 0人預約,國立臺東大學 館藏查詢系統,...,C031122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,1.0,E227675,NaN,873.57 857 89 c.4,圖書/一般,書在館,NaN,NaN,不可預約,國立金門大學圖書館,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,2.0,E219874,NaN,873.57 857 89 c.3,圖書/一般,書在館,NaN,NaN,不可預約,國立金門大學圖書館,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,3.0,E178789,NaN,873.57 857 89 c.2,圖書/一般,書在館,NaN,NaN,不可預約,國立金門大學圖書館,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# webpac_jsp_crawler(
#     url="http://192.192.231.232/webpacIndex.jsp",
#     ISBN="9789573317241")